# Surface Temperature Comparison - Daily Averages

### Import the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set()

### Import the necessary data, look at the general schema. The units of the upwelling data (BS_final & RCP_final) are Degrees Celsius. I have done some calculations beforehand to reduce the time waiting on calculations, and saved data to excel files.

In [ ]:
LandMask = xr.open_dataset('../Data Samples/NetCDF/LandMask.nc').mask[0,:,:]
maskpoints=pd.read_excel('../Data Samples/Excels/maskpointsfile.xlsx')
BS = pd.read_excel('../Data Samples/Excels/BS_ST_DayofYear_LatLonAvg.xlsx')
RCP = pd.read_excel('../Data Samples/Excels/RCP_ST_DayofYear_LatLonAvg.xlsx')
BS.head()

#### Schema is Year, Day, and 10 Model Ensemble. Forward fill the years and group the index by year and day.

In [ ]:
BS[['Year']] = BS[['Year']].ffill(axis=0)
RCP[['Year']] = RCP[['Year']].ffill(axis=0)
BS.set_index(['Year','Day'],inplace=True)
RCP.set_index(['Year', 'Day'], inplace=True)
RCP = RCP.loc[2010:2050]

# Plotting

### Set the desired models for plotting in the "models" variable below. See column headers above for potential model names.

### The following plots will show each models daily average upwelling volume for each year for the Historical (1965-2005, Blue) and Future (2010-2050, Red), as well as the averages across both time periods (thick lines).

In [ ]:
# Example:
models = ['ACCESS', 'BCC-CSM', 'CCSM4','FGOALS', 'GFDL', 'IPSL', 'MIROC5', 'MPI', 'NorESM']

In [ ]:
for model in models:
    model_df = BS.loc[:, model]
    Year = 1965
    fig = plt.figure(figsize=(20,20))
    ax1 = fig.add_subplot(111)
    while Year <=2005:
        year_df = model_df.loc[Year]
        plot = ax1.plot(year_df, linewidth=0.5, label="_nolegend_", color='cornflowerblue', linestyle=':')
        Year+=1
    total_ensemble_BS = model_df.groupby('Day').mean()
    
    model_df = RCP.loc[:, model]
    Year = 2010
    while Year <=2050:
        year_df = model_df.loc[Year]
        plot = ax1.plot(year_df, linewidth=0.5, label="_nolegend_", color='firebrick', linestyle=':')
        Year+=1
    total_ensemble_RCP = model_df.groupby('Day').mean()
    plot = ax1.plot(total_ensemble_BS, linewidth=6.0, label='Historical Average', color='cornflowerblue')
    plot = ax1.plot(total_ensemble_RCP, linewidth=6.0, label='Future Average',color='firebrick')
    fig.suptitle('{} Daily Mean Surface Temperature: Historical & Future'.format(model), fontsize=32)
    leg=ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, fontsize=12)
    xlabel = ax1.set_xlabel('Day of Year', fontsize=30)
    ylabel = ax1.set_ylabel('Temperature (K)', fontsize=30)
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0, box.width * 0.8, box.height])

### The following block creates an average over all models (aka "ensemble average"). Generate the Mann-Whitney U Statistic, P value, and effective size to determine the statistical significance of the difference between the two datasets.

In [ ]:
import scipy.stats as stats
Year = 1965
fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(111)
while Year <=2005:
    year_df = BS.loc[Year].mean(axis=1)
    plot = ax1.plot(year_df, linewidth=0.25, label='Ensemble',color='cornflowerblue')
    Year+=1
total_ensemble_BS = BS.groupby('Day').mean().mean(axis=1)
plot = ax1.plot(total_ensemble_BS, linewidth=5.0, label='Ensemble',color='cornflowerblue')

Year = 2010
while Year <=2050:
    year_df = RCP.loc[Year].mean(axis=1)
    plot = ax1.plot(year_df, linewidth=0.25, label='Ensemble',color='firebrick')
    Year+=1
total_ensemble_RCP = RCP.groupby('Day').mean().mean(axis=1)
plot = ax1.plot(total_ensemble_RCP, linewidth=5.0, label='Ensemble',color='firebrick')
fig.suptitle('Ensemble Daily Mean Surface Temperature: Historical & Future', fontsize=32)
ax1.set_xlabel('Day of Year', fontsize=30)
ax1.set_ylabel('Temperature (K)', fontsize=30)


na = len(total_ensemble_BS)
nb = len(total_ensemble_RCP)
U, p = stats.mannwhitneyu(total_ensemble_BS, total_ensemble_RCP)
r = 1-(2*U)/(na*nb)
stats =  pd.DataFrame([[U,p,r]], columns=['U statistic', 'p Value', 'Effective Size'])
stats